# Calendar heatmaps with Bokeh

In [2]:
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.palettes import OrRd
from bokeh.plotting import figure
from bokeh.transform import transform
from bokeh.models import ColumnDataSource, LinearColorMapper, Range1d, SingleIntervalTicker, Panel, Tabs
from datetime import datetime
import numpy as np
from pathlib import Path
output_notebook()

Loading BokehJS ...

In [24]:
def plot_cal(
    df, 
    date_column,
    color_column,
    mode='calendar',
    nan_color='white',
    color_low_value=None,
    color_high_value=None,
    hover_tooltips=None,
    text_color='grey',
    text_font='courier',
    palette=OrRd[9][::-1],
    weekdays=None,
    xaxis_major_label_orientation='horizontal',
    yaxis_major_label_orientation='horizontal',
    fig_args={
        'plot_width':300,
        'plot_height':200,
        'toolbar_location':None,
        'tools':'hover',
        'x_axis_location':"above"
    },
    rect_args={
        'width':1,
        'height':1,
        'line_color':'white',
        'line_width':0,
    },
    show_dates=True
):
    """
    Function making calendar heatmap plots using Bokeh.

    :param df: pandas DataFrame with date_column in datetime format
    :param date_column: name of the date column
    :param color_column: name of the column to use for heatmap colors
    :param mode: "github" for github contribution like calendar, "calendar" for standard calendar layout
    :param nan_color: color for NaN values of color_column
    :param palette: heatmap color palette
    :param color_low_value: low value to map to color palette
    :param color_high_value: high value to map to color palette
    :param hover_tooltips: bokeh hover tooltips
    :param text_color: text color for axis labels
    :param text_font: text font size for axis labels
    :param weekdays: weekday label ordering e.g. ['Mon', 'Tue'..., 'Sun']
    :param xaxis_major_label_orientation: "vertical" or "horizontal"
    :param yaxis_major_label_orientation: "vertical" or "horizontal"
    :param fig_args: Bokeh figure() args
    :param rect_args: Bokeh Rect() args
    :param show_dates: bool; show calendar dates on plot
    :return: plot obj, plot column data source obj
    """
    
    x = 'day'
    y = 'week'
    day_of_month_column = 'dom'
    
    df['date_str_abdY'] = df[date_column].dt.strftime('%a %b %d, %Y')
    df['date_str_Ymd'] = df[date_column].dt.strftime('%Y-%m-%d')
    df['month'] = df[date_column].dt.strftime('%b')
    df['year'] = df[date_column].dt.strftime('%y')
    df[x] = df[date_column].dt.strftime('%a')
    df[day_of_month_column] = df[date_column].dt.day
    df[y] = df[date_column].dt.isocalendar().week
    
    # Get absolute week numbering for the input data date range
    d1 = dict(df.groupby('year').max()[y].cumsum())
    d2 = {str(int(k)+1):v for k,v in d1.items()}
    d2[min(d1.keys())] = 0    
    df[y] = df.apply(lambda row: row[y] + d2[row['year']], axis=1)
    
    # Monotonize week numbers 
    # e.g. pd.to_datetime('2019-12-30').isocalendar() --> (2020,1,1)
    # For visualization, instead of counting this as 1st week of 2020, 
    # this should be last week of 2019
    df[y] = df.sort_values('date').groupby(['month', 'year'])[[y]].apply(lambda x: np.maximum.accumulate(x))
    
    if not color_low_value:
        color_low_value = df[color_column].min()
        
    if not color_high_value:
        color_high_value = df[color_column].min()
    
    source = ColumnDataSource(df)
    mapper = LinearColorMapper(
        palette=palette, 
        low=df[color_column].min(), 
        high=df[color_column].max(),
        nan_color=nan_color
    )

    if mode == 'calendar':
        if not weekdays:
            weekdays=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
            
        range_args ={
            'x_range':weekdays,
            'y_range':Range1d(float(df[y].max())+0.5, float(df[y].min())-0.5)#list(reversed([i(i) for i in df[y].unique()]))
        }
        xy = {'x':x, 'y':y}

    elif mode == 'github':
        if not weekdays:
            weekdays=list(reversed(['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']))

        range_args ={
            'x_range':Range1d(float(df[y].max())+0.5, float(df[y].min())-0.5), #Range1d(-1,53),  #[i(i) for i in df[y].unique()],
            'y_range':weekdays
        }
        xy = {'x':y, 'y':x}
    
    p = figure(**{**fig_args, **range_args})

    rect_renderer = p.rect(
        **{**rect_args, **xy}, 
        source=source, 
        fill_color=transform(color_column, mapper)
    )

    if show_dates:
        text_renderer = p.text(
            **xy, 
            text=day_of_month_column,
            text_align='center',
            text_baseline='middle',
            text_color=text_color,
            text_font=text_font,
            source=source
        )

    if range_args['x_range'] != weekdays:
        p.xaxis.ticker = SingleIntervalTicker(interval=1)

    if range_args['y_range'] != weekdays:
        p.yaxis.ticker = SingleIntervalTicker(interval=1)

    labels = {}
    for k,v in dict(df.groupby(['month', 'year'])[y].min()).items():
        if k[0] == 'Jan':
            # Add year annotation for each January in data
            labels[int(v)]=f"'{k[1]} {k[0]}"
        else:
            labels[int(v)]=k[0]
    
    for i in df[y].unique():
        if i not in labels.keys():
            labels[int(i)] = ''
            
    p.axis.major_label_overrides = labels
    p.xaxis.major_label_orientation = xaxis_major_label_orientation 
    p.yaxis.major_label_orientation = yaxis_major_label_orientation 
    p.axis.major_label_text_color=text_color
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.outline_line_color = None
    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.minor_tick_line_color = None
    p.axis.major_label_standoff = 0
    p.hover.tooltips = [('Date','@date_str_abdY')] + hover_tooltips
    rect_renderer.nonselection_glyph.fill_alpha=1

    return p, source


# Generate data

In [25]:
df = pd.DataFrame(
    {'date': pd.date_range('2019-11-30','2020-11-30')}
)
df['val'] = np.random.rand(len(df))

# Calendar plots

In [29]:
p1, _ = plot_cal(
    df, 
    date_column='date', 
    color_column='val',
    mode='github', 
    fig_args={
        'plot_width':900,
        'plot_height':150,
        'tools':'hover',
        'toolbar_location':None,
        'x_axis_location':"above"
    }, 
    rect_args={
        'width':1,
        'height':1,
        'line_color':'grey',
        'line_width':1,
    },
    hover_tooltips=[('Value','@val')], 
    show_dates=False
)


In [32]:
df_cal = df.loc[(
        df['date'] > (datetime.today() - pd.Timedelta('61 days'))
   ) & (df['date'] < (datetime.today() - pd.Timedelta('31 days')))
].copy()

p2, _ = plot_cal(
    df_cal, 
    date_column='date', 
    color_column='val',
    mode='calendar',
    yaxis_major_label_orientation='vertical',
    fig_args={
        'plot_width':400,
        'plot_height':200,
        'tools':'hover',
        'toolbar_location':None,
        'x_axis_location':"above",
        'y_axis_location':"left",
    }, 
    rect_args={
        'width':1,
        'height':1,
        'line_color':'grey',
        'line_width':1,
    },
    hover_tooltips=[('Value','@val')], 
    show_dates=True
)


In [33]:
show(p1)
show(p2)